In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/retail-store-inventory-and-demand-forecasting/sales_data.csv


## **Preliminary Data Cleaning Hypotheses**

### **1. Missing Data & Integrity Checks**
- **Null Values**:  
  - Columns like `Weather Condition`, `Competitor Pricing`, and `Discount` may contain missing data.  
  - **Action**: Impute using forward-fill or median values based on `Category`/`Region`.  
- **Inventory-Sales Mismatch**:  
  - If `Units Sold` > `Inventory Level`, data is invalid (sales cannot exceed stock).  
  - **Action**: Flag/remove such records.  

### **2. Temporal & Categorical Issues**
- **Date Gaps**:  
  - Check for missing dates (e.g., store closures during `Epidemic=1`).  
  - **Action**: Resample time series and forward-fill static features (e.g., `Store ID`).  
- **Categorical Consistency**:  
  - `Category` or `Region` may have typos (e.g., "Electronics" vs. "Eletronics").  
  - **Action**: Standardize labels using fuzzy matching.  

### **3. Outliers & Anomalies**
- **Demand Spikes**:  
  - Extreme `Units Sold` during `Promotion=1` or `Epidemic=1`.  
  - **Action**: Winsorize or cap values at 99th percentile.  
- **Negative Inventory**:  
  - `Inventory Level` < 0 suggests data entry errors.  
  - **Action**: Set to zero or treat as missing.  

### **4. Feature Engineering Ideas**
- **Time Features**:  
  - Derive `Day_of_week`, `Is_holiday`, and `Lag_7_Demand` from `Date`.  
- **Competitor Impact**:  
  - Create `Price_ratio = Price / Competitor Pricing` to measure relative affordability.  
- **Stockout Risk**:  
  - `Stock_coverage = Inventory Level / (7-day avg Units Sold)`.



In [2]:
# Imports and Configuration
from pathlib import Path
import sys

DATA_PATH = Path('/kaggle/input/retail-store-inventory-and-demand-forecasting/sales_data.csv')
CHUNKSIZE = 50_000
DATE_COL = 'Date'

# Optimized dtypes
DTYPES = {
    'Store ID': 'category',
    'Product ID': 'category',
    'Category': 'category',
    'Region': 'category',
    'Inventory Level': 'uint16',
    'Units Sold': 'uint16',
    'Units Ordered': 'uint16',
    'Price': 'float32',
    'Discount': 'float32',
    'Weather Condition': 'category',
    'Promotion': 'bool',
    'Competitor Pricing': 'float32',
    'Seasonality': 'category',
    'Epidemic': 'bool',
    'Demand': 'uint16'
}

In [3]:
# Data Import Function with Row Tracking
def import_retail_data():
    chunks = []
    error_log = []
    stats = {
        'total_rows': 0,
        'rows_dropped': {
            'date_parsing': 0,
            'inventory_adjustment': 0,
            'sales_adjustment': 0,
            'dtype_conversion': 0,
            'chunk_failures': 0
        }
    }
    
    try:
        with pd.read_csv(
            DATA_PATH,
            chunksize=CHUNKSIZE,
            dtype=DTYPES,
            parse_dates=[DATE_COL],
            on_bad_lines='warn',
            encoding='utf-8'
        ) as reader:
            
            for chunk_idx, chunk in enumerate(reader):
                stats['total_rows'] += len(chunk)
                try:
                    # Validate required columns
                    required_cols = set(DTYPES.keys())
                    missing_cols = required_cols - set(chunk.columns)
                    if missing_cols:
                        raise ValueError(f"Missing columns: {missing_cols}")
                    
                    # Track original row count
                    original_rows = len(chunk)
                    
                    # Clean data - Date parsing
                    chunk[DATE_COL] = pd.to_datetime(chunk[DATE_COL], errors='coerce')
                    date_dropped = chunk[DATE_COL].isna().sum()
                    chunk = chunk.dropna(subset=[DATE_COL])
                    stats['rows_dropped']['date_parsing'] += date_dropped
                    
                    # Clean data - Inventory adjustment
                    neg_inventory = (chunk['Inventory Level'] < 0).sum()
                    chunk['Inventory Level'] = chunk['Inventory Level'].clip(lower=0)
                    stats['rows_dropped']['inventory_adjustment'] += neg_inventory
                    
                    # Clean data - Sales adjustment
                    bad_sales = (chunk['Units Sold'] > chunk['Inventory Level']).sum()
                    chunk['Units Sold'] = np.where(
                        chunk['Units Sold'] > chunk['Inventory Level'],
                        chunk['Inventory Level'],
                        chunk['Units Sold']
                    )
                    stats['rows_dropped']['sales_adjustment'] += bad_sales
                    
                    # Handle numeric overflows
                    for col in ['Inventory Level', 'Units Sold', 'Units Ordered']:
                        try:
                            chunk[col] = pd.to_numeric(chunk[col], downcast='unsigned')
                        except:
                            pass
                    
                    chunks.append(chunk)
                    
                except Exception as e:
                    stats['rows_dropped']['chunk_failures'] += len(chunk)
                    error_log.append(f"Chunk {chunk_idx} failed: {str(e)}")
                    continue
        
        if not chunks:
            raise ValueError("No valid data chunks were processed")
            
        # Combine and finalize
        df = pd.concat(chunks, ignore_index=False)
        
        # Final dtype enforcement and tracking
        for col, dtype in DTYPES.items():
            if col in df.columns:
                try:
                    df[col] = df[col].astype(dtype)
                except Exception as e:
                    error_log.append(f"Dtype conversion failed for {col}: {str(e)}")
                    # Count rows that would fail conversion
                    mask = pd.to_numeric(df[col], errors='coerce').isna()
                    stats['rows_dropped']['dtype_conversion'] += mask.sum()
                    df[col] = pd.to_numeric(df[col], errors='coerce')
        
        df = df.set_index(DATE_COL).sort_index()
        stats['final_rows'] = len(df)
        
        return df, error_log, stats
        
    except Exception as e:
        error_log.append(f"Fatal import error: {str(e)}")
        return None, error_log, stats

In [4]:
# Execute and Display Enhanced Results
# Execute the import
df, import_errors, import_stats = import_retail_data()

# Display results
if df is not None:
    print(f"Successfully imported {import_stats['final_rows']:,} rows (out of {import_stats['total_rows']:,} total)")
    print("\nRows dropped during processing:")
    for reason, count in import_stats['rows_dropped'].items():
        print(f"- {reason.replace('_', ' ').title()}: {count:,} rows")
    
    print("\nSample data:")
    display(df.head())
else:
    print("Import failed")

if import_errors:
    print("\nEncountered warnings/errors:")
    for i, error in enumerate(import_errors, 1):
        print(f"{i}. {error}")

Successfully imported 76,000 rows (out of 76,000 total)

Rows dropped during processing:
- Date Parsing: 0 rows
- Inventory Adjustment: 0 rows
- Sales Adjustment: 0 rows
- Dtype Conversion: 0 rows
- Chunk Failures: 0 rows

Sample data:


,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Price,Discount,Weather Condition,Promotion,Competitor Pricing,Seasonality,Epidemic,Demand
Date,,,,,,,,,,,,,,,
2022-01-01,S001,P0001,Electronics,North,195,102,252,72.720001,5.0,Snowy,False,85.730003,Winter,False,115
2022-01-01,S001,P0002,Clothing,North,117,117,249,80.160004,15.0,Snowy,True,92.019997,Winter,False,229
2022-01-01,S001,P0003,Clothing,North,247,114,612,62.939999,10.0,Snowy,True,60.080002,Winter,False,157
2022-01-01,S001,P0004,Electronics,North,139,45,102,87.629997,10.0,Snowy,False,85.190002,Winter,False,52
2022-01-01,S001,P0005,Groceries,North,152,65,271,54.410000,0.0,Snowy,False,51.630001,Winter,False,59


## **Retail Data Insights Overview**

### **1. Investigation Purpose**
Systematically evaluate dataset quality to:
- Identify integrity issues affecting forecast accuracy  
- Uncover demand/sales/inventory patterns  
- Guide model selection based on data characteristics  
- Document critical metadata for reproducibility  

### **2. Key Investigation Areas**

#### **A. Data Quality Checks**
**Rationale**: Ensure reliable inputs for forecasting  
**Targets**:  
- **Category labels**: Detect typos/inconsistencies (e.g., "Electronis" vs "Electronics")  
- **Date validity**: Identify gaps or anomalous timestamps  
- **Demand outliers**: Flag artificial caps or placeholder values  
- **Missing data**: Quantify gaps in key columns (weather/pricing)  

#### **B. Dataset Dimensions**
**Rationale**: Understand data granularity and coverage  
**Metrics**:  
| Dimension         | Purpose                                  |
|-------------------|-----------------------------------------|
| Product Count     | Determine SKU-level vs category forecasting |  
| Category Count    | Identify overbroad groupings            |
| Date Range        | Assess seasonality coverage            |  
| Store/Region      | Evaluate geographical representation   |

#### **C. Demand Characteristics**  
**Rationale**: Inform model selection  
**Analysis**:  
- Value range (min/max) to detect censoring  
- Zero-demand frequency for intermittent-demand handling  
- Epidemic-period variance for shock resilience  

### **3. Insights-to-Action Mapping**

| Finding                  | Impact                                                         |
|--------------------------|----------------------------------------------------------------|
| High zero-demand %       | Use Croston's/TSB models                                      |
| Regional sales clusters  | Add region dummy variables                                    |  
| Epidemic demand spikes   | Implement shock indicator features                           |
| Category label typos     | Consolidate labels pre-analysis                               |
| Missing weather data     | Impute regionally or exclude weather-dependent models        |

### **4. Target Outputs**
1. **Data Quality Scorecard**  
   - Priority cleaning tasks  
   - Model compatibility assessment  
2. **Feature Engineering Roadmap**  
3. **Model Selection Justification**